In [4]:
def imap_login(address=None, password=None):
    '''Wrapper for logging into to email through IMAP
    
    ARGS: 
    address - str (defaul: None, prompt input). Email address 
    being connected to.
    
    password - str (default: None, prompt input). Password for email address.
    
    Returns:
    Mail object connected to corresponding server for email address'''
    
    import imaplib
    
    if not address:
        address = input('Enter you email address: ')
        
    if not password:
        password = input('Enter your password')
        
    if 'gmail' in address:
        SMTP_SERVER = 'imap.gmail.com'
    elif 'yahoo' in password:
        SMTP_SERVER = 'imap.mail.yahoo.com'
    else:
        raise NameError('Please enter a gmail or yahoo email address')
        
    SMTP_PORT = 993
    try:
        mail = imaplib.IMAP4_SSL(SMTP_SERVER)
        mail.login(address, password)
    except Exception as e:
        raise
        
    return mail

In [10]:
def search_mailbox(mail, inbox='inbox'):
    """Connects to mailbox and collects a list of ids from mailbox
    
    ARGS:
    mail - logged in mail object
    
    inbox - str (defauls: 'inbox'). Mailbox to connect to. Must be valid
    imap mailbox.
    
    Returns:
    tup (mail object, list of mail_ids)
    If you don't need the ids, you can use an underscore like so:
    mail, _ = search_mailbox(mail)"""
    
    import imaplib
    
    mail.select(inbox)
    
    typ, data = mail.search(None, 'ALL')
    
    mail_ids = data[0].decode()
    mail_ids = mail_ids.split()
    
    return mail, mail_ids

In [13]:
def print_mail(mail, i_d=None, mail_part='(RFC822)'):
    """Prints out mail messages to screen
    
    ARGS:
    mail - logged in mail object
    
    i_d - single id or list of ids
    id(s) of email(s) to print
    
    Returns: None; prints message to screen"""
    
    import imaplib
    import email
    
    if type(i_d) == 'list':
        for i in i_d:
            typ, data = mail.fetch(str(i).encode(), mail_part)
            meta = email.message_from_string(data[0][1].decode())
            print(meta)
    
    else:
        typ, data = mail.fetch(str(i_d).encode(), mail_part)
        meta = email.message_from_string(data[0][1].decode())
        print(meta)

In [75]:
def save_mail(mail, i_d, filename='email_data.csv', verbose=False):
    """Writes email data to csv
    
    ARGS: 
    mail - logged in mail object
    
    i_d - list of i_ds
    ids of messages to get
    
    filename - string ending in .csv (default: 'email_data.csv')
    name of file to write to 
    
    Returns: None, saves data to csv"""
    
    import imaplib
    import email
    import csv
    
    csv_file = open(filename, 'w', encoding='UTF-8')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['id', 'uid', 'from_', 'subject', 'msg', 'content_type'])
    
    for i in i_d:
        try:
            typ, data = mail.fetch(str(i).encode(), '(UID RFC822)')

            uid = email.message_from_bytes(data[0][0])
            uid = uid.get_payload()
            uid = uid.split()[-3]

            meta = email.message_from_bytes(data[0][1])
            from_ = meta['From']
            subject = meta['Subject']
            content_type = meta['Content-Type'].split(';')[0]
            
            msg = meta.get_payload()
            if type(msg) == list:
                msg = msg[0].get_payload()
            print(i)
            if verbose:
                print('UID: ', uid)
                print('From: ', from_)
                print('Subject: ', subject)
                print('Content-Type: ', content_type)
                print('Message: ', msg)
            csv_writer.writerow([i, uid, from_, subject, msg, content_type])
            print('Message saved')
        except Exception as e:
            print(e)

#### Testing above func

In [182]:
save_mail(mail, mail_ids[10000:], filename='email_data_10k+.csv')

10001
Message saved
10002
Message saved
10003
Message saved
10004
Message saved
10005
Message saved
10006
Message saved
10007
Message saved
10008
Message saved
10009
Message saved
10010
Message saved
10011
Message saved
10012
Message saved
10013
Message saved
10014
Message saved
10015
Message saved
10016
Message saved
10017
Message saved
10018
Message saved
10019
Message saved
10020
Message saved
10021
Message saved
10022
Message saved
10023
Message saved
10024
Message saved
10025
Message saved
10026
Message saved
10027
Message saved
10028
Message saved
10029
Message saved
10030
Message saved
10031
Message saved
10032
Message saved
10033
Message saved
10034
Message saved
10035
Message saved
10036
Message saved
10037
Message saved
10038
Message saved
10039
Message saved
10040
Message saved
10041
Message saved
10042
Message saved
10043
Message saved
10044
Message saved
10045
Message saved
10046
Message saved
10047
Message saved
10048
Message saved
10049
Message saved
10050
Message saved


In [144]:
import pandas as pd

emails = pd.read_csv('email_data.csv')

print(emails.shape)

emails.head()

(997, 6)


,id,uid,from_,subject,msg,content_type
0,1,1,Gmail Team <mail-noreply@google.com>,Get Gmail on your mobile phone,[image: Access Gmail on your mobile phone]\r\...,multipart/alternative
1,2,2,Gmail Team <mail-noreply@google.com>,Import your contacts and old email,You can import your contacts and mail from Yah...,multipart/alternative
2,3,3,Gmail Team <mail-noreply@google.com>,Customize Gmail with colors and themes,"To spice up your inbox with colors and themes,...",multipart/alternative
3,4,4,"""PokerStars Support"" <support@pokerstars.com>",Email Validation Code,"\r\nDear YAMGJJ,\r\n\r\nYour email validation ...",text/plain
4,5,5,"""Cindy Cooper"" <customercare@reimage.com>",Welcome to Reimage News and Specials,<HTML><HEAD><META HTTP-EQUIV=3D'Content-Type' ...,text/html


In [157]:
bberry = emails.iloc[21]['msg']

In [161]:
bberry = ' '.join(bberry.split(r'<p>'))
' '.join(bberry.split('\r\n'))

'Congratulations AvrahamJaih, A BlackBerry&#174; ID with this email address has been successfully created. BlackBerry&#174; ID makes managing your BlackBerry&#174; services easier. Learn more about your BlackBerry&#174; ID by visiting: <a href=http://www.blackberry.com/blackberryid>www.blackberry.com/blackberryid</a>. If you did not create or do not recognize this account, please click <a href=https://blackberryid.blackberry.com/bbid/deactivate?accountid=66239020862743463651350204028279073521&locale=ZW4tVVM%3D&key=SARCLBKGREMBMPRGUOTPDQRXDUCWKZUV>here</a> to delete this BlackBerry ID. --------------------------------------------------------------------- <br> This transmission (including any attachments) may contain confidential information, privileged material (including material protected by the solicitor-client or other applicable privileges), or constitute non-public information. Any use of this information by anyone other than the intended recipient is prohibited. If you have recei

In [148]:
for idx, row in emails.iterrows():
    print(idx)
    print(row['msg'])

0
 [image: Access Gmail on your mobile phone]
<http://www.google.com/intl/en/mobile/default/mail.html>

The days of needing your computer to get to your inbox are long gone. You
can now use Gmail on your mobile phone to access your email from anywhere.
       Get Gmail for your phone
=BB<http://www.google.com/intl/en/mobile/default/mail.html#utm_source=3Dwel=
-eml&utm_medium=3Deml&utm_campaign=3Den>

1
You can import your contacts and mail from Yahoo!, Hotmail, AOL, and many
other web mail or POP accounts. If you want, we'll even keep importing your
mail for the next 30 days.
     Import contacts and mail
=BB<https://mail.google.com/mail/#settings/accounts>

We know it can be a pain to switch email accounts, and we hope this makes
the transition to Gmail a bit easier.

- The Gmail Team

Please note that importing is not available if you're using Internet
Explorer 6.0. To take advantage of the latest Gmail features, please upgrad=
e
to a fully supported
browser<http://mail.google.com/su

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




973
Free shipping on stylish chill chasers. 

This is an advertising message from Nordstrom.com.

To view this e-mail online, click here
http://e.nordstrom.com/a/tBOnaO3AHxQ8rB8eAYqNspV$-Vr/vonline?t=BOnaO3AHxQ8rB8eAYqNspV$-Vr&email=avraham.jacobsohn@gmail.com&EMAIL_SKEY_1=19796914


Free Shipping. Free Returns. Every Day. See Details.*


Fabled Cables

From fishermen to fashion runways, cable knits have epitomized
cold-weather style. Update your sweater wardrobe with a classic in
the season's newest colors.

SHOP:

SWEATERS
http://e.nordstrom.com/a/tBOnaO3AHxQ8rB8eAYqNspV$-Vr/sweaters?EMAIL_SKEY_1=19796914

NEW ARRIVALS
http://e.nordstrom.com/a/tBOnaO3AHxQ8rB8eAYqNspV$-Vr/newarrive?EMAIL_SKEY_1=19796914

Find sought-after sweaters in petite sizes.

SHOP PETITE SWEATERS
http://e.nordstrom.com/a/tBOnaO3AHxQ8rB8eAYqNspV$-Vr/petites?EMAIL_SKEY_1=19796914


Born to Be Warm

Combine iconic style with the coziest shearling on earth. The result:
UGG Australia.

SHOP:

UGG AUSTRALIA
http://e.

In [99]:
emails['content_type'].unique()

array(['multipart/alternative', 'text/plain', 'text/html',
       'multipart/mixed', 'multipart/related', 'Multipart/Alternative'],
      dtype=object)

In [101]:
emails['content_type'].value_counts()

multipart/alternative    422
text/html                318
text/plain               220
multipart/mixed           22
multipart/related         13
Multipart/Alternative      2
Name: content_type, dtype: int64

In [102]:
emails['content_type'] = emails['content_type'].map(lambda x: 'multipart/alternative' if x == 'Multipart/Alternative' else x)

emails['content_type'].unique()

array(['multipart/alternative', 'text/plain', 'text/html',
       'multipart/mixed', 'multipart/related'], dtype=object)

In [103]:
emails['content_type'].value_counts()

multipart/alternative    424
text/html                318
text/plain               220
multipart/mixed           22
multipart/related         13
Name: content_type, dtype: int64

In [107]:
mask = emails['content_type'] == 'multipart/alternative'

ma = emails[mask]

print(ma.shape)

ma.head()

(424, 6)


,id,uid,from_,subject,msg,content_type
0,1,1,Gmail Team <mail-noreply@google.com>,Get Gmail on your mobile phone,[image: Access Gmail on your mobile phone]\r\...,multipart/alternative
1,2,2,Gmail Team <mail-noreply@google.com>,Import your contacts and old email,You can import your contacts and mail from Yah...,multipart/alternative
2,3,3,Gmail Team <mail-noreply@google.com>,Customize Gmail with colors and themes,"To spice up your inbox with colors and themes,...",multipart/alternative
31,32,32,support@fulltiltpoker.com,=?UTF-8?B?RW1haWwgVmFsaWRhdGlvbiBDb2Rl?=,Welcome to Full Tilt Poker thekill21.\r\nBefor...,multipart/alternative
34,35,35,cashier@fulltiltpoker.com,=?UTF-8?B?RnVsbCBUaWx0IFBva2VyIC0gSW1wb3J0YW50...,"Dear Avraham Jacobsohn,\r\n\r\nThank you for y...",multipart/alternative


In [110]:
emails.isnull().sum()

id               0
uid              0
from_            0
subject         19
msg              5
content_type     0
dtype: int64

In [111]:
emails = emails.dropna(subset=['msg'])

In [113]:
emails[emails['msg'].str.contains('https')].iloc[0]['msg']

"You can import your contacts and mail from Yahoo!, Hotmail, AOL, and many\r\nother web mail or POP accounts. If you want, we'll even keep importing your\r\nmail for the next 30 days.\r\n     Import contacts and mail\r\n=BB<https://mail.google.com/mail/#settings/accounts>\r\n\r\nWe know it can be a pain to switch email accounts, and we hope this makes\r\nthe transition to Gmail a bit easier.\r\n\r\n- The Gmail Team\r\n\r\nPlease note that importing is not available if you're using Internet\r\nExplorer 6.0. To take advantage of the latest Gmail features, please upgrad=\r\ne\r\nto a fully supported\r\nbrowser<http://mail.google.com/support/bin/answer.py?answer=3D6557&hl=3Den&=\r\nutm_source=3Dwel-eml&utm_medium=3Deml&utm_campaign=3Den>\r\n.\r\n"

In [115]:
emails[emails['msg'].str.contains('https')].iloc[5]['msg']

'<HTML><HEAD><META HTTP-EQUIV=3D\'Content-Type\' CONTENT=3D\'text/html; cha=\r\nrset=3Dwindows-1250\'></HEAD><BODY dir=3Dltr><DIV dir=3Dltr><div align=3D=\r\n"center" style=3D"font: normal 11px Arial; color: black;" dir=3D"ltr">I=\r\nf you cannot see this email properly, please click <a href=3D"http://ww=\r\nw=2Eweb-view=2Enet/Show/0XC59FF2CB7C422199A973FDF29C942928E94BC2477C37C=\r\n640B22E81D505B26271=2Ehtm" style=3D"font: normal 11px Arial;color: blue=\r\n;">here</a></div><table width=3D"600" cellspacing=3D"0" cellpadding=3D"=\r\n0" border=3D"0" align=3D"center">\r\n    <tbody>\r\n        <tr>\r\n            <td style=3D"padding-bottom: 33px;">\r\n            <div style=3D"height: 3px; line-height: 3px; font-size: 1px=\r\n; border-bottom: 1px solid rgb(220, 220, 220); background: none repeat =\r\nscroll 0% 0% rgb(197, 197, 197);">&nbsp;</div>\r\n            </td>\r\n        </tr>\r\n        <tr>\r\n            <td style=3D"padding-bottom: 32px;">\r\n            <table width=3D"100%" 

In [142]:
mask = emails['msg'].str.startswith('<HTML')
emails = emails[~mask]

mask = emails['msg'].str.startswith('<html')
emails = emails[~mask]

In [143]:
emails[emails['msg'].str.startswith('<')]

,id,uid,from_,subject,msg,content_type
155,156,156,"""Adam"" <agrunwerg@hotmail.com>",Mosesbet.com December Newsletter,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",multipart/mixed
157,158,158,"""=?UTF-8?Q?Full=20Tilt=20Poker=20Support?="" <s...",=?UTF-8?Q?Update=20regarding=20recent=20connec...,"<head> <meta http-equiv=3D""Cont...",text/html
160,161,161,UB <support@cs.ub.com>,"THEKILL21, welcome to UB","<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",multipart/alternative
162,163,163,UB <support@cs.ub.com>,Welcome to UB | Sign-up bonus,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",multipart/alternative
170,171,171,UB <news@members.ub.com>,UB | Getting Started,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",multipart/alternative
...,...,...,...,...,...,...
986,990,991,"""Bank of America Alert"" <onlinebanking@ealerts...",Bank of America Alert: Available Balance,"<!DOCTYPE html PUBLIC ""-//W3C//DTD HTML 4.01 T...",text/html
987,991,992,"""Bank of America Alert"" <onlinebanking@ealerts...",Bank of America Alert: Available Balance,"<!DOCTYPE html PUBLIC ""-//W3C//DTD HTML 4.01 T...",text/html
990,994,995,"""Bank of America Alert"" <onlinebanking@ealerts...",Bank of America Alert: Available Balance,"<!DOCTYPE html PUBLIC ""-//W3C//DTD HTML 4.01 T...",text/html
991,995,996,"""Bank of America Alert"" <onlinebanking@ealerts...",Bank of America Alert: Available Balance,"<!DOCTYPE html PUBLIC ""-//W3C//DTD HTML 4.01 T...",text/html


In [126]:
_, data = mail.fetch(str(5).encode(), '(UID RFC822)')
len(data)

2

In [132]:
len(data[0][0])
data[0][0]
data[0][1]

b'Delivered-To: avraham.jacobsohn@gmail.com\r\nReceived: by 10.231.15.9 with SMTP id i9cs140559iba;\r\n        Mon, 11 Oct 2010 12:01:00 -0700 (PDT)\r\nReceived: by 10.223.75.208 with SMTP id z16mr2064586faj.125.1286823659278;\r\n        Mon, 11 Oct 2010 12:00:59 -0700 (PDT)\r\nReturn-Path: <reply@activetrail.com>\r\nReceived: from mmmailgw35.mymarketing.co.il (mmmailgw35.mymarketing.co.il [194.90.201.35])\r\n        by mx.google.com with SMTP id 13si1765206fah.34.2010.10.11.12.00.58;\r\n        Mon, 11 Oct 2010 12:00:59 -0700 (PDT)\r\nReceived-SPF: pass (google.com: domain of reply@activetrail.com designates 194.90.201.35 as permitted sender) client-ip=194.90.201.35;\r\nAuthentication-Results: mx.google.com; spf=pass (google.com: domain of reply@activetrail.com designates 194.90.201.35 as permitted sender) smtp.mail=reply@activetrail.com\r\nReceived: from mmmailgw35.mymarketing.co.il ([194.90.201.35]) by activetrail.com (activetrail.com)\r\nX-BBounce: 72385335|526314|Avraham.Jacobsohn

In [124]:
msg = email.message_from_bytes(data[0][1])
msg.get_payload()

'<HTML><HEAD><META HTTP-EQUIV=3D\'Content-Type\' CONTENT=3D\'text/html; cha=\r\nrset=3Dwindows-1250\'></HEAD><BODY dir=3Dltr><DIV dir=3Dltr><div align=3D=\r\n"center" style=3D"font: normal 11px Arial; color: black;" dir=3D"ltr">I=\r\nf you cannot see this email properly, please click <a href=3D"http://ww=\r\nw=2Eweb-view=2Enet/Show/0XC59FF2CB7C422199A4F246DE72C2D2C06BDA9DDAACE21=\r\n598B22E81D505B26271=2Ehtm" style=3D"font: normal 11px Arial;color: blue=\r\n;">here</a></div><br /> <table cellspacing=3D"0" cellpadding=3D"0" alig=\r\nn=3D"center" width=3D"600">     <tbody>         <tr>             <td st=\r\nyle=3D"padding-bottom: 33px;"><div style=3D"height: 3px; line-height: 3=\r\npx; font-size: 1px; border-bottom: 1px solid rgb(220, 220, 220); backgr=\r\nound: none repeat scroll 0% 0% rgb(197, 197, 197);">&nbsp;</div></td>  =\r\n       </tr>         <tr>             <td style=3D"padding-bottom: 32px=\r\n;"><table cellspacing=3D"0" cellpadding=3D"0" width=3D"100%">          =\r\n      

In [ ]:
import numpy as np



In [ ]:
print_mail()

In [178]:
mail = imap_login('', '')

In [179]:
mail, mail_ids = search_mailbox(mail)

In [87]:
print_mail(mail, 31947)

Delivered-To: avrahamjacobsohn@gmail.com
Received: by 2002:a4f:d50a:0:0:0:0:0 with SMTP id l10csp246861ivm;
        Fri, 15 Nov 2019 04:40:27 -0800 (PST)
X-Google-Smtp-Source: APXvYqzfIkh87WqbKTXh2ppKzlzQcSvz8pvaKXBm5k28mNITEZXjJLSjI47QpcsO1EzJbvz4+bzq
X-Received: by 2002:a92:17ce:: with SMTP id 75mr536098ilx.88.1573821627036;
        Fri, 15 Nov 2019 04:40:27 -0800 (PST)
ARC-Seal: i=1; a=rsa-sha256; t=1573821627; cv=none;
        d=google.com; s=arc-20160816;
        b=f5Of9T2dZaHqBwToX1oz5YbjS1+ICsIRbNvgD+ic6fV5J6MnNvoz8VFGjzS05B3IJj
         oWZUjP8CBZw15wCkBRSac1Duxf847FouZgadwJxtpSWvCIsaZCR2UK/e6vZRUQymr0+U
         pBNSrfEhYOwPDe1hR5FFToQiyto7pmYa8OMbxek9fzOWdt20M3VBpLsUyyAnikqS27s5
         6Fyno37+rwvHWnINi0/DIO1i061F8luzFAzaqNDaDDtiFldJCXFW2wsPyBXIZBVfyB6y
         oSoxdlzO6+vRVUVX5sJgQMwY3Nk0bCTPilr8hNoBu/P7gDELXs86evYqcWuTJR+yI6VA
         BL2w==
ARC-Message-Signature: i=1; a=rsa-sha256; c=relaxed/relaxed; d=google.com; s=arc-20160816;
        h=content-transfer-encoding:sub

In [166]:
_, data = mail.fetch(str(130).encode(), '(UID RFC822)')

In [176]:
meta = email.message_from_bytes(data[0][1])

meta.get_payload()[0].get_payload()[0].get_payload()

'=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=\r\n=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D\r\n=3D PokerTracker - Order Confirmation             =3D\r\n=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=\r\n=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D\r\n\r\nPayment Method: Credit Card\r\nOrder Date: December 12, 2010 07:44 EST\r\nConfirmation Code: PT-E76B57AEFFB33EC2\r\n\r\nPT3 Holdem: Full Version | Qty: 1 | Price: $89.99TableTracker - Bronze Su=\r\nbscription* | Monthly | Price: -TOTAL: $89.99 USD\r\n\r\n* The first month of TableTracker is free with a new PokerTracker 3 purch=\r\nase.  You will not be billed until the end of the first month at which po=\r\nint you will be charged the normal subscription rate.  Cancel online at a=\r\nny time via the link below:\r\n\r\nhttp://www.pokertracker.com/products/TT/cancel.php\r\n\r\n\r\n=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=3D=

In [63]:
import email
import imaplib

typ, data = mail.fetch(str(mail_ids[0]).encode(), '(RFC822)')
meta = email.message_from_bytes(data[0][1])

In [67]:
meta.get_payload()[0].get_payload()

' [image: Access Gmail on your mobile phone]\r\n<http://www.google.com/intl/en/mobile/default/mail.html>\r\n\r\nThe days of needing your computer to get to your inbox are long gone. You\r\ncan now use Gmail on your mobile phone to access your email from anywhere.\r\n       Get Gmail for your phone\r\n=BB<http://www.google.com/intl/en/mobile/default/mail.html#utm_source=3Dwel=\r\n-eml&utm_medium=3Deml&utm_campaign=3Den>\r\n'

In [57]:
meta.keys()

['MIME-Version',
 'Received',
 'Date',
 'Message-ID',
 'Subject',
 'From',
 'To',
 'Content-Type']

In [34]:
for key in meta.keys():
    print('Key: ', key)
    print('Value: ', meta[key])

Key:  MIME-Version
Value:  1.0
Key:  Received
Value:  by 10.231.15.9; Mon, 11 Oct 2010 10:20:49 -0700 (PDT)
Key:  Date
Value:  Mon, 11 Oct 2010 10:20:49 -0700
Key:  Message-ID
Value:  <AANLkTikC_TbfmsohwVMUSsRg8XBVtw6g5pTdPi0SD-Vu@mail.gmail.com>
Key:  Subject
Value:  Get Gmail on your mobile phone
Key:  From
Value:  Gmail Team <mail-noreply@google.com>
Key:  To
Value:  Avraham Jacobsohn <avraham.jacobsohn@gmail.com>
Key:  Content-Type
Value:  multipart/alternative; boundary=00032555f0eeeb34d404925a9497


In [28]:
meta.get_payload()[0].get_payload()

' [image: Access Gmail on your mobile phone]\r\n<http://www.google.com/intl/en/mobile/default/mail.html>\r\n\r\nThe days of needing your computer to get to your inbox are long gone. You\r\ncan now use Gmail on your mobile phone to access your email from anywhere.\r\n       Get Gmail for your phone\r\n=BB<http://www.google.com/intl/en/mobile/default/mail.html#utm_source=3Dwel=\r\n-eml&utm_medium=3Deml&utm_campaign=3Den>\r\n'

In [73]:
typ, data = mail.fetch(str(mail_ids[3]).encode(), '(UID RFC822)')
meta = email.message_from_string(data[0][1].decode())
meta.keys()

['Delivered-To',
 'Received',
 'Received',
 'Return-Path',
 'Received',
 'Received-SPF',
 'Authentication-Results',
 'Received',
 'DKIM-Signature',
 'X-PS-ID',
 'Received',
 'Date',
 'From',
 'To',
 'Subject',
 'Message-Id',
 'MIME-Version',
 'Content-Type',
 'Content-Transfer-Encoding']

In [74]:
meta.get_payload()

'\r\nDear YAMGJJ,\r\n\r\nYour email validation code is 3338\r\n\r\nTo validate your email address simply click on the following link: http=\r\n://www.pokerstars.com/locate/pokerstars/validate/3338\r\n\r\nAlternatively you may also validate your email by following these steps:\r\n\r\nFirst return to your PokerStars program.\r\n\r\nGo to the main lobby screen (the first screen that appears when you sta=\r\nrt your program), and select "Account" from the top menu.\r\n\r\nChoose the "Validate Email Address ..." menu item. You will be prompted=\r\n to LogIn (unless you are already logged in). Enter the UserID (nicknam=\r\ne) and Password that you provided when creating your account.\r\n\r\nAfter completing the login process you will get the "Validate email add=\r\nress" dialog box. Enter your validation code and click on the "Validate=\r\n" button. A message should appear confirming that your email address ha=\r\ns been validated.\r\n\r\nThis will activate your account and you can now enjoy

In [44]:
msg = meta.get_payload()[0].get_payload()
meta['Content-Type']
#''.join(msg.split('\r\n'))

'multipart/alternative;\r\n\tboundary="----=_NextPart_000_021C_01D59B93.4A57BF20"'

In [49]:
uid = email.message_from_string(data[0][0].decode())

In [55]:
uid.get_payload().split()[-3]

'32680'